In [1]:
# Data prep
import os
import pandas as pd
import numpy as  np
import sqlite3
import warnings
import random

from random import randrange
from datetime import timedelta, datetime

warnings.filterwarnings('ignore')

### Users

In [2]:
users = pd.read_csv('prepared/users.csv')
users

,Unnamed: 0,id,name,address,postalcode,city,birthday,phone,country,active,excluded,age
0,0,1,Breena Connow,16 Acker Alley,NaN,Villa Serrano,1978-12-31,722-967-3477,Bolivia,True,False,44
1,1,2,Harris Dunkerton,8356 Talisman Crossing,99871,Inari,1990-12-03,909-642-7516,Finland,True,False,32
2,2,3,Merrie Grindle,3 Homewood Junction,NaN,Cengjia,1944-10-13,876-111-3766,China,False,True,78
3,3,4,Arlan Morstatt,05 Straubel Place,13315-000,Cabreúva,1945-03-03,870-897-0217,Brazil,True,False,77
4,4,5,Kiele Dedrick,947 Mallard Drive,3515-517,Bodiosa a Velha,1950-02-26,879-743-0896,Portugal,True,False,72
...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,996,Jere Matys,34 Park Meadow Crossing,195047,Florencia,1960-01-06,967-680-6203,Colombia,True,False,62
996,996,997,Bax Levick,17468 Kropf Court,NaN,Gaotang,1961-12-21,399-742-6576,China,True,False,61
997,997,998,Hermine Bowerbank,4100 Brown Avenue,32-060,Rzozów,1989-12-06,240-137-7616,Poland,False,True,33
998,998,999,Beaufort Caulier,72724 Rigney Hill,11320,Kandana,1999-02-13,961-971-2453,Sri Lanka,False,True,23


In [3]:
f = open("populate.sql", "w", encoding="utf-8")

for index, row in users.iterrows():
    f.write("INSERT INTO \"User\" VALUES ('" 
            + str(row['id']) + "','" 
            + str(row['name']) + "','"
            + str(row['address']) + "','"
            + str(row['postalcode']) + "','"
            + str(row['city']) + "','"
            + str(row['birthday']) + "','"
            + str(row['age']) + "','"
            + str(row['phone']) + "','"
            + str(row['country']) + "','"
            + str(row['active']) + "','"
            + str(row['excluded']) + "');\n")
f.write('\n')
f.close()

### Events

In [4]:
baseball = pd.read_csv('prepared/americanfootball.csv')
basketball = pd.read_csv('prepared/basketball.csv')
football = pd.read_csv('prepared/football.csv')
hockey = pd.read_csv('prepared/hockey.csv')
events = football.append(baseball).append(basketball).append(hockey)
events = events.drop(columns=['Unnamed: 0', 'ID'])
events = events.sort_values(by='Date')
events = events.rename(columns={"Score": "Result"})
events['Result'] = events['Result'].apply(lambda x: str(x).replace("–", "-"))
events.loc[events["Active"] == True, "Result"] = np.NaN
events.loc[events["Result"] == 'nan', "Result"] = np.NaN
events.insert(0, 'ID', range(0, 0 + len(events)))
events = events.reset_index(drop=True)
events

,ID,Category,Date,Result,Name,Active
0,0,Football,2022-08-05,4-0,Benfica VS Arouca,False
1,1,Football,2022-08-05,2-1,Lyon VS Ajaccio,False
2,2,Football,2022-08-05,0-2,Crystal Palace VS Arsenal,False
3,3,Football,2022-08-05,1-6,Eint Frankfurt VS Bayern Munich,False
4,4,Football,2022-08-06,0-5,Clermont Foot VS Paris S G,False
...,...,...,...,...,...,...
4663,4663,Football,2023-04-08,NaN,Leeds United VS Crystal Palace,True
4664,4664,Football,2023-04-08,NaN,Aston Villa VS Nott ham Forest,True
4665,4665,Football,2023-04-08,NaN,Fulham VS West Ham,True
4666,4666,Football,2023-04-08,NaN,Bochum VS Stuttgart,True


In [5]:
f = open("populate.sql", "a", encoding="utf-8")

for index, row in events.iterrows():
    f.write("INSERT INTO \"Event\" VALUES ('" 
            + str(row['ID']) + "','" 
            + str(row['Name']) + "','"
            + str(row['Date']) + "','"
            + str(row['Category']) + "','"
            + str(row['Active']) + "','"
            + str(row['Result']) + "');\n")
f.write('\n')
f.close()

### Market

In [6]:
eventIDs = []
for i in range(3000):
    eventIDs.append(random.randint(0,4075))

markets = pd.DataFrame(eventIDs, columns =['EventID'])
markets.insert(0, 'ID', range(0, 0 + len(markets)))
markets['Type'] = np.random.choice(['Score', 'Winner'], len(markets))
markets

,ID,EventID,Type
0,0,3236,Score
1,1,1190,Score
2,2,1268,Winner
3,3,1020,Winner
4,4,920,Winner
...,...,...,...
2995,2995,1455,Winner
2996,2996,2018,Score
2997,2997,2254,Winner
2998,2998,3734,Winner


In [7]:
f = open("populate.sql", "a", encoding="utf-8")

for index, row in markets.iterrows():
    f.write("INSERT INTO \"Market\" VALUES ('" 
            + str(row['ID']) + "','" 
            + str(row['Type']) + "','"
            + str(row['EventID']) + "');\n")
f.write('\n')
f.close()

### Contract

In [8]:
marketIDs = []
for i in range(3000):
    marketIDs.append(random.randint(0,2999))

contracts = pd.DataFrame(marketIDs, columns =['MarketID'])
contracts['Type'] = np.random.choice(['Home', 'Draw', 'Away'], len(contracts))
contracts.insert(0, 'ID', range(0, 0 + len(contracts)))
contracts

,ID,MarketID,Type
0,0,1919,Home
1,1,342,Draw
2,2,1581,Away
3,3,27,Home
4,4,2338,Home
...,...,...,...
2995,2995,1494,Draw
2996,2996,547,Away
2997,2997,815,Home
2998,2998,2437,Draw


In [9]:
f = open("populate.sql", "a", encoding="utf-8")

for index, row in contracts.iterrows():
    f.write("INSERT INTO \"Contract\" VALUES ('" 
            + str(row['ID']) + "','" 
            + str(row['Type']) + "','"
            + str(row['MarketID']) + "');\n")
f.write('\n')
f.close()

### Order

In [10]:
userIDs = []
contractsIDs = []

nonexcludedUsers = users[users['excluded'] == False]
activeUsers = nonexcludedUsers[nonexcludedUsers['active'] == True]

activeUsersIDs = activeUsers["id"].values.tolist()

for i in range(3000):
    userIDX = random.randint(1,(len(activeUsersIDs)-1))
    userIDs.append(activeUsersIDs[userIDX])
    contractsIDs.append(random.randint(0,2999))
    
orders = pd.DataFrame(userIDs, columns =['userID'])
orders['contractID'] = contractsIDs
orders.insert(0, 'ID', range(0, 0 + len(orders)))

orders

,ID,userID,contractID
0,0,2,2820
1,1,742,2007
2,2,424,1933
3,3,610,367
4,4,261,2869
...,...,...,...
2995,2995,828,334
2996,2996,576,1565
2997,2997,253,2107
2998,2998,875,2449


In [11]:
contractsInOrders = contracts[contracts['ID'].isin(contractsIDs)]
marketIDS = contractsInOrders["MarketID"].values.tolist()
contractsInOrders

,ID,MarketID,Type
1,1,342,Draw
2,2,1581,Away
6,6,2315,Away
7,7,1846,Draw
8,8,2380,Home
...,...,...,...
2994,2994,2562,Away
2996,2996,547,Away
2997,2997,815,Home
2998,2998,2437,Draw


In [12]:
eventsInMarkets = markets[markets['ID'].isin(marketIDs)]
eventsInMarkets = pd.merge(eventsInMarkets, events, left_on="EventID", right_on="ID")
eventsInMarkets = eventsInMarkets.drop(columns = ['ID_y'])
eventsInMarkets.rename(columns = {'ID_x':'MarketID'}, inplace = True)
eventsInMarkets

,MarketID,EventID,Type,Category,Date,Result,Name,Active
0,0,3236,Score,Football,2023-02-01,NaN,Lyon VS Brest,True
1,1488,3236,Score,Football,2023-02-01,NaN,Lyon VS Brest,True
2,2,1268,Winner,Football,2022-11-01,2-4,Viktoria Plzeň VS Barcelona,False
3,3,1020,Winner,American Football,2022-10-24,33 - 14,Chicago Bears - New England Patriots,False
4,4,920,Winner,Basketball,2022-10-21,113 - 111,Portland Trail Blazers - Phoenix Suns,False
...,...,...,...,...,...,...,...,...
1864,2991,77,Score,Football,2022-08-14,3-2,Fiorentina VS Cremonese,False
1865,2992,3175,Winner,Football,2023-01-29,NaN,Casa Pia VS Santa Clara,True
1866,2993,2226,Score,Basketball,2022-12-16,NaN,Los Angeles Lakers - Denver Nuggets,True
1867,2996,2018,Score,Basketball,2022-12-04,112 - 122,Detroit Pistons - Memphis Grizzlies,False


In [13]:
status = []
for index, row in orders.iterrows():
    marketid = contracts.loc[contracts['ID'] == row['contractID'], 'MarketID'].iloc[0]
    result = str(eventsInMarkets.loc[eventsInMarkets['MarketID'] == marketid, 'Result'].iloc[0])
   
    if result == np.nan or result == 'nan':
        status.append('Pending')
    else:
        values = str(result).split('-')
        bet = contractsInOrders.loc[contractsInOrders['ID'] == row['contractID'], 'Type'].iloc[0]
        if bet == 'Draw':
            if int(values[0]) == int(values[1]):
                status.append('Won')
            else: 
                status.append('Lost')
        elif bet == 'Home':
            if int(values[0]) > int(values[1]):
                status.append('Won')
            else: 
                status.append('Lost')
        else:
            if int(values[0]) < int(values[1]):
                status.append('Won')
            else: 
                status.append('Lost')
print(status)

['Lost', 'Lost', 'Pending', 'Lost', 'Pending', 'Lost', 'Won', 'Pending', 'Pending', 'Lost', 'Pending', 'Pending', 'Won', 'Pending', 'Won', 'Lost', 'Pending', 'Pending', 'Won', 'Pending', 'Won', 'Pending', 'Pending', 'Lost', 'Pending', 'Won', 'Pending', 'Pending', 'Lost', 'Pending', 'Lost', 'Lost', 'Won', 'Pending', 'Lost', 'Won', 'Won', 'Pending', 'Lost', 'Pending', 'Lost', 'Pending', 'Lost', 'Won', 'Lost', 'Won', 'Lost', 'Won', 'Pending', 'Won', 'Pending', 'Lost', 'Lost', 'Pending', 'Lost', 'Pending', 'Lost', 'Pending', 'Pending', 'Pending', 'Pending', 'Pending', 'Pending', 'Lost', 'Lost', 'Won', 'Lost', 'Pending', 'Pending', 'Lost', 'Pending', 'Lost', 'Pending', 'Lost', 'Lost', 'Pending', 'Lost', 'Won', 'Won', 'Won', 'Lost', 'Pending', 'Lost', 'Lost', 'Pending', 'Lost', 'Lost', 'Lost', 'Pending', 'Pending', 'Pending', 'Pending', 'Won', 'Pending', 'Won', 'Pending', 'Lost', 'Pending', 'Lost', 'Lost', 'Pending', 'Pending', 'Lost', 'Pending', 'Lost', 'Pending', 'Pending', 'Pending', 'Pen

In [14]:
orders['Status'] = status
orders['Status'].value_counts()

Pending    1481
Lost       1017
Won         502
Name: Status, dtype: int64

In [15]:
initialValues = []
currentValues = []

for i in range(3000):
    initialValues.append(random.randint(1,200))

for i in range(3000):
    currentValues.append(random.randint(0,initialValues[i]))

In [16]:
orders['initialValue'] = initialValues
orders['currentValue'] = currentValues
orders

,ID,userID,contractID,Status,initialValue,currentValue
0,0,2,2820,Lost,69,1
1,1,742,2007,Lost,74,32
2,2,424,1933,Pending,61,41
3,3,610,367,Lost,11,8
4,4,261,2869,Pending,30,9
...,...,...,...,...,...,...
2995,2995,828,334,Pending,154,111
2996,2996,576,1565,Pending,20,8
2997,2997,253,2107,Pending,123,8
2998,2998,875,2449,Pending,57,48


### Deal

In [17]:
t_order = orders
t_events = events
t_contracts = contracts

temp = eventsInMarkets[['MarketID', 'EventID']]

merged1 = pd.merge(temp, t_contracts, left_on="MarketID", right_on="MarketID")
merged1 = merged1.rename(columns={'ID': 'ContractID'})

data = pd.merge(merged1, t_order, left_on="ContractID", right_on="contractID")
data = data.rename(columns={'ID': 'OrderID'})
data

,MarketID,EventID,ContractID,Type,OrderID,userID,contractID,Status,initialValue,currentValue
0,0,3236,161,Away,1022,155,161,Pending,40,33
1,0,3236,161,Away,2574,369,161,Pending,182,137
2,0,3236,866,Home,2041,592,866,Pending,96,42
3,0,3236,866,Home,2990,607,866,Pending,177,89
4,1488,3236,2753,Draw,1772,913,2753,Pending,43,42
...,...,...,...,...,...,...,...,...,...,...
2995,2992,3175,1657,Home,1151,87,1657,Pending,99,22
2996,2992,3175,1657,Home,1572,63,1657,Pending,148,57
2997,2992,3175,2953,Draw,70,684,2953,Pending,10,3
2998,2992,3175,2953,Draw,327,506,2953,Pending,104,80


In [18]:
data.sort_values(by=['ContractID'])

,MarketID,EventID,ContractID,Type,OrderID,userID,contractID,Status,initialValue,currentValue
469,342,897,1,Draw,358,862,1,Lost,57,24
470,342,897,1,Draw,2149,835,1,Lost,196,158
1853,1581,372,2,Away,2364,915,2,Lost,23,17
47,2315,3243,6,Away,1396,347,6,Pending,113,57
48,2315,3243,6,Away,2222,822,6,Pending,197,190
...,...,...,...,...,...,...,...,...,...,...
1045,815,3644,2997,Home,1124,718,2997,Pending,58,21
2620,2437,3582,2998,Draw,2373,449,2998,Pending,89,25
2619,2437,3582,2998,Draw,1935,847,2998,Pending,91,31
2406,2146,1982,2999,Home,616,918,2999,Won,61,46


In [19]:
def getOppositeBet(bet):
    if bet == 'Away':
        return 'Home'
    elif bet == 'Home':
        return 'Away'
    return 'Draw'

def getValue(init, curr):
    return init - curr

def random_date(start, end):
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds=random_second)

def getDate(eventID):
    #print(eventID)
    startDate = datetime.strptime('2022-01-01', "%Y-%m-%d")
    #print(startDate)
    eventDate = datetime.strptime(str(events.at[eventID, 'Date']), "%Y-%m-%d")
    date = random_date(startDate, eventDate)
    return date.strftime('%Y-%m-%d')

def getUser(userID):
    while True:
        idx = random.randint(1,(len(activeUsersIDs)-1))
        if activeUsersIDs[idx] != int(userID):
            return activeUsersIDs[idx]    
        
def getStatus(status):
    if status == 'Won':
        return 'Lost'
    elif status == 'Lost':
        return 'Won'
    return 'Pending'

In [20]:
dictOrders = {'ID': [], 'userID': [], 'contractID': [], 'Status': [], 'initialValue': [], 'currentValue': []}
dictDeals = {'orderBuy': [], 'orderSell': [], 'value':[], 'date': []} 
#orderBuy = [] orderSell = [] value = [] date = []
orderIDCurr = 3000

for index, row in data.iterrows():
    newType = getOppositeBet(str(row['Type']))
    newValue = getValue(int(row['initialValue']), int(row['currentValue']))
    dictDeals['orderBuy'].append(int(row['OrderID']))
    dictDeals['orderSell'].append(orderIDCurr)
    dictDeals['value'].append(newValue)
    dictDeals['date'].append(getDate(int(row['EventID'])))
    dictOrders['ID'].append(orderIDCurr)
    dictOrders['userID'].append(getUser(int(row['userID'])))
    dictOrders['contractID'].append(int(row['contractID']))
    dictOrders['Status'].append(getStatus(str(row['Status'])))
    dictOrders['initialValue'].append(newValue)
    dictOrders['currentValue'].append(0)
    orderIDCurr+=1
    
print(len(dictOrders['ID']))

3000


In [21]:
addedOrders = pd.DataFrame.from_dict(dictOrders)
addedOrders

,ID,userID,contractID,Status,initialValue,currentValue
0,3000,441,161,Pending,7,0
1,3001,277,161,Pending,45,0
2,3002,752,866,Pending,54,0
3,3003,333,866,Pending,88,0
4,3004,258,2753,Pending,1,0
...,...,...,...,...,...,...
2995,5995,623,1657,Pending,77,0
2996,5996,331,1657,Pending,91,0
2997,5997,796,2953,Pending,7,0
2998,5998,105,2953,Pending,24,0


In [22]:
finalOrders = orders.append(addedOrders)
finalOrders

,ID,userID,contractID,Status,initialValue,currentValue
0,0,2,2820,Lost,69,1
1,1,742,2007,Lost,74,32
2,2,424,1933,Pending,61,41
3,3,610,367,Lost,11,8
4,4,261,2869,Pending,30,9
...,...,...,...,...,...,...
2995,5995,623,1657,Pending,77,0
2996,5996,331,1657,Pending,91,0
2997,5997,796,2953,Pending,7,0
2998,5998,105,2953,Pending,24,0


In [23]:
f = open("populate.sql", "a", encoding="utf-8")

for index, row in finalOrders.iterrows():
    f.write("INSERT INTO \"Order\" VALUES ('" 
            + str(row['ID']) + "','" 
            + str(row['userID']) + "','"
            + str(row['contractID']) + "','" 
            + str(row['initialValue']) + "','"
            + str(row['currentValue']) + "','" 
            + str(row['Status']) + "');\n")
f.write('\n')
f.close()

In [24]:
deals = pd.DataFrame.from_dict(dictDeals)
deals

,orderBuy,orderSell,value,date
0,1022,3000,7,2022-08-03
1,2574,3001,45,2022-06-24
2,2041,3002,54,2022-07-30
3,2990,3003,88,2022-08-22
4,1772,3004,1,2022-09-11
...,...,...,...,...
2995,1151,5995,77,2022-05-04
2996,1572,5996,91,2023-01-07
2997,70,5997,7,2022-05-13
2998,327,5998,24,2022-07-25


In [25]:
f = open("populate.sql", "a", encoding="utf-8")

for index, row in deals.iterrows():
    f.write("INSERT INTO \"Deal\" VALUES ('" 
            + str(row['orderBuy']) + "','" 
            + str(row['orderSell']) + "','"
            + str(row['value']) + "','" 
            + str(row['date']) + "');\n")
f.write('\n')
f.close()